# This is a draft

In [2]:
import sys
import os
import requests
import json
import pandas as pd
import myvariant

In [3]:
# Input
#user_input = input('Please input file PATH. ')
CSV_input = '/Users/manolodominguez/Desktop/git-repos/STP_mini_projects/Igor-Manuel/Igor-Manuel/csv_file.csv'

# input to pd data frame
df = pd.read_csv(CSV_input) 

# An extra copy will be neccesary for manipulation.
df_draft = pd.read_csv(CSV_input) 

### Here we read and found what we want from the CSV

In [30]:
##### Reading CSV file values and looking for variants IDs ######

# Find Variant ID (rs000000) in CSV
# \d+ is neccesary in case the line find a rs+something. rs\d+ looks for rs+ numbers
rs = df_draft[df_draft.apply(lambda x:x.str.contains("rs\d+"))].dropna(how='all').dropna(axis=1, how='all')

# Now, we save the results found in a dict key=index and value=variand ID
if rs.empty == False:
    ind = rs.index.to_list()
    vals = list(rs.stack().values)
    row2rs = dict(zip(ind, vals))
    print(row2rs)
    # We need to remove the row where rs has been found to avoid repetitions
    # In case in same row more than one kind of ID Variant is stored (i.e rs# and NM_#)   
    for index, rs  in row2rs.items(): 
        # This will be done in df_draft
        df_draft = df_draft.drop(index)
    
## Same thing with other ID variants

# Here with Variant ID (NM_0000000) in CSV
NM = df_draft[df_draft.apply(lambda x:x.str.contains("NM_\d+"))].dropna(how='all').dropna(axis=1, how='all')

if NM.empty == False:
    ind = NM.index.to_list()
    vals = list(NM.stack().values)
    row2NM = dict(zip(ind, vals))
    print(row2NM)
    
    for index, NM  in row2NM.items(): 
        df_draft = df_draft.drop(index)

# Here with Variant ID (NP_0000000) in CSV
NP = df_draft[df_draft.apply(lambda x:x.str.contains("NP_\d+"))].dropna(how='all').dropna(axis=1, how='all')

if NP.empty == False:
    ind = NP.index.to_list()
    vals = list(NP.stack().values)
    row2NP = dict(zip(ind, vals))
    print(row2NP)
    
    for index, NP  in row2NP.items(): 
        df_draft = df_draft.drop(index)

# Here with ClinVar field (RCV#) in CSV
RCV = df_draft[df_draft.apply(lambda x:x.str.contains("RCV\d+"))].dropna(how='all').dropna(axis=1, how='all')
    
if RCV.empty == False:
    ind = RCV.index.to_list()
    vals = list(RCV.stack().values)
    row2RCV = dict(zip(ind, vals))
    print(row2RCV)
    
    for index, NP  in row2NP.items(): 
        df_draft = df_draft.drop(index)


#https://rest.ensembl.org/documentation/info/vep_hgvs_get
# If you see the output of this input you will agree with me that this is one common problem 

{11: 'rs268', 56: 'rs56116432', 57: 'rs334'}


In [31]:
row2rs

{11: 'rs268', 56: 'rs56116432', 57: 'rs334'}

### Here we use API for requesting

In [19]:
# Ensemble REST API
server = "http://rest.ensembl.org"

#Empty dict
rs2features ={}

for index, rs  in row2rs.items(): 
    
    # Rows where substring 'rs' has been found need to be delete to avoid repetition
    # This will be done in df_draft
    df_draft = df_draft.drop(index)
    
    # Request each rs found
    ext = "/variation/human/{}?".format(rs)
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    decoded = r.json()

    # Dict of dict
    rs2features[rs] = decoded
# We need to filter this data. This data can be used to fill the ultimate VCF
# Or can be used also to verify that the values of the CSV are ook
# For example gene name or coodinates
    
# This step made here with variant ID must be made with ... 

In [20]:
rs2features

{'rs268': {'MAF': 0.005192,
  'most_severe_consequence': 'missense_variant',
  'clinical_significance': ['uncertain significance',
   'pathogenic',
   'risk factor'],
  'synonyms': ['VCV000001550',
   'RCV000001615',
   'RCV000988041',
   'RCV000781944',
   'VAR_004239',
   '609708.0033',
   'NM_000237.3:c.953A>G',
   'NP_000228.1:p.Asn318Ser',
   'NM_000237.2:c.953A>G',
   'rs52818902',
   'rs386571803',
   'rs17850737'],
  'evidence': ['Frequency',
   '1000Genomes',
   'Cited',
   'ESP',
   'Phenotype_or_Disease',
   'ExAC',
   'TOPMed',
   'gnomAD'],
  'source': 'Variants (including SNPs and indels) imported from dbSNP',
  'mappings': [{'assembly_name': 'GRCh38',
    'strand': 1,
    'start': 19956018,
    'end': 19956018,
    'seq_region_name': '8',
    'location': '8:19956018-19956018',
    'coord_system': 'chromosome',
    'allele_string': 'A/G',
    'ancestral_allele': 'A'}],
  'name': 'rs268',
  'ambiguity': 'R',
  'minor_allele': 'G',
  'var_class': 'SNP'},
 'rs56116432': {'ma

In [6]:
mv = myvariant.MyVariantInfo()

In [7]:
mv.getvariant('chr17:g.7578532A>G', fields = 'clinvar')

{'_id': 'chr17:g.7578532A>G',
 '_version': 4,
 'clinvar': {'_license': 'http://bit.ly/2SQdcI0',
  'allele_id': 27396,
  'alt': 'G',
  'chrom': '17',
  'cytogenic': '17p13.1',
  'gene': {'id': '7157', 'symbol': 'TP53'},
  'hg19': {'end': 7578532, 'start': 7578532},
  'hg38': {'end': 7675214, 'start': 7675214},
  'hgvs': {'coding': ['LRG_321t1:c.398T>C',
    'LRG_321t2:c.398T>C',
    'LRG_321t3:c.398T>C',
    'LRG_321t4:c.398T>C',
    'LRG_321t5:c.2T>C',
    'LRG_321t6:c.2T>C',
    'LRG_321t7:c.2T>C',
    'LRG_321t8:c.281T>C',
    'NM_000546.5:c.398T>C',
    'NM_001126112.2:c.398T>C',
    'NM_001126113.2:c.398T>C',
    'NM_001126114.2:c.398T>C',
    'NM_001126115.1:c.2T>C',
    'NM_001126116.1:c.2T>C',
    'NM_001126117.1:c.2T>C',
    'NM_001126118.1:c.281T>C',
    'NM_001276695.2:c.281T>C',
    'NM_001276696.2:c.281T>C',
    'NM_001276697.2:c.-80T>C',
    'NM_001276698.2:c.-80T>C',
    'NM_001276699.2:c.-80T>C',
    'NM_001276760.2:c.281T>C',
    'NM_001276761.2:c.281T>C'],
   'genomic'